In [52]:
import win32com.client

body_content = ""  # Объявление глобальной переменной

def move_mail_with_draft_reply(from_folder_name, to_folder_name, alternate_account):
    global body_content  # Указание, что мы используем глобальную переменную
    
    # Создаём экземпляр приложения Outlook
    outlook = win32com.client.Dispatch("Outlook.Application")
    namespace = outlook.GetNamespace("MAPI")

    # Находим исходную папку
    root_folder = namespace.Folders.Item(1)
    from_folder = find_subfolder(root_folder, from_folder_name)

    # Находим целевую папку
    to_folder = find_subfolder(root_folder, to_folder_name)

    # Перемещаем первое письмо из исходной папки в целевую
    if from_folder and to_folder:
        try:
            first_item = from_folder.Items.GetFirst()
            body_content = first_item.Body
            # Создаем черновик ответа на письмо
            reply = first_item.ReplyAll()
            reply.Subject = f"Re: {first_item.Subject}"  # Сохраняем тему письма
            # Оставляем оригинальное письмо в теле ответа и добавляем наш ответ
            reply.HTMLBody = f"""<p>Здравствуйте!<br>
             <br>
            В течение 5 рабочих дней рассмотрим Ваш запрос.<br>
             <br>
            С уважением,<br>
            Маркелова (Воробьева) Евгения Олеговна<br>
            Эксперт<br>
            ПАО Сбербанк, ПЦП Подразделение центрального подчинения "Банк рядом"<br>
            Территориальное управление сетью УС<br>
            Среднерусский банк УРМ Группа Тула<br>
            Тел.: 8-800-700-5000 (вн. 622160)<br>
            Россия, Тула, Комсомольская, д. 31<br>
            Даем людям уверенность и надежность!<br>
            www.sberbank.ru<br>
            {reply.HTMLBody}</p>"""
            reply.SentOnBehalfOfName = alternate_account  # Устанавливаем альтернативный аккаунт для отправки
            reply.Display(True)  # Показываем окно для редактирования
            # Перемещаем оригинальное письмо
            first_item.Move(to_folder)
            print("Письмо успешно перемещено, и открыт черновик ответа.")
        except Exception as e:
            print(f"Ошибка при перемещении письма или создании черновика: {e}")
    else:
        print("Одна из указанных папок не найдена.")

# Отправка письма на внутреннюю почту


def find_subfolder(parent_folder, folder_name):
    """Функция для поиска подпапки по имени."""
    for folder in parent_folder.Folders:
        if folder.Name == folder_name:
            return folder
    return None


# Использование
alternate_account = "Retail_ATM@sberbank.ru"
move_mail_with_draft_reply("Проект", "Отработанные", alternate_account)

# Можно вывести значение глобальной переменной
print(body_content)


Письмо успешно перемещено, и открыт черновик ответа.
ВНИМАНИЕ: ВНЕШНИЙ ОТПРАВИТЕЛЬ 
Подозрительное письмо? Перешли на 'SCST@sberbank.ru' 	

Добрый день.

Прошу Вас рассмотреть возможность размещения банкомата Сбербанк в ММ Трафарет,
по адресу: 410005, Саратовская обл, Саратов г, Большая Горная ул, дом № 157.
Заранее спасибо. 

 

 

-- 

С уважением,

Чернова Елена Валерьевна

Менеджер по коммерческой недвижимости

Отдел экспансии Саратовского филиала

АО «Тандер»

тел.: 8-927-156-84-06

г. Энгельс, ул. Промышленная, 16

Семья магазинов «Магнит»

magnit-info.ru





 



In [53]:
# обрезаем из текста подпись

def truncate_text(text, delimiter="С уважением,"):
    index = text.find(delimiter)
    if index != -1:
        truncated_text = text[:index].strip()
        return truncated_text
    return text

original_text = body_content
truncated_text = truncate_text(original_text)
print(truncated_text)

ВНИМАНИЕ: ВНЕШНИЙ ОТПРАВИТЕЛЬ 
Подозрительное письмо? Перешли на 'SCST@sberbank.ru' 	

Добрый день.

Прошу Вас рассмотреть возможность размещения банкомата Сбербанк в ММ Трафарет,
по адресу: 410005, Саратовская обл, Саратов г, Большая Горная ул, дом № 157.
Заранее спасибо. 

 

 

--


In [54]:
# Определяем адрес из текста письма

from natasha import (
    Segmenter,
    MorphVocab,
    LOC,
    AddrExtractor
)

# Импортируем дополнительные модули для работы с регулярными выражениями
import re

# Создаем словарь для регионов, включая республики
regions = {
    'республика': ['Башкортостан', 'Татарстан', 'Удмуртия'],
    'область': ['Московская область', 'Ленинградская область'],
    'край': ['Краснодарский край', 'Ставропольский край']
}

def extract_region(address):
    # Функция для извлечения региона из строки адреса
    for region_type, region_list in regions.items():
        for region_name in region_list:
            if region_name.lower() in address.lower():
                return f"{region_type.capitalize()} {region_name}"
    return None

# Основной блок программы
segmenter = Segmenter()
morph_vocab = MorphVocab()
addr_extractor = AddrExtractor(morph_vocab)

# Текст, содержащий адрес
text = truncated_text

# Извлечение адреса из текста
matches = addr_extractor(text)
facts = [i.fact.as_json for i in matches]

# Цикл для вывода адреса в удобной форме
for fact in facts:
    address_parts = list(fact.values())  # Разбираем адрес на части
    region = extract_region(' '.join(address_parts))  # Извлекаем регион
    
     # Объединение всех значений в одну строку
single_line = ' '.join([item for sublist in facts for item in sublist.values()])
print(single_line)

410005 индекс Саратовская область Саратов Большая Горная улица № 157 дом


In [55]:
# Импортируем дополнительные модули для работы с регулярными выражениями
import re
import requests

# Ваш API ключ для Яндекс Геокодера
api_key = 'e7e22a42-0d2d-4b2b-9641-7dc3cddc1c94'

# Адрес для поиска
address = single_line

# Формирование URL запроса
base_url = 'https://geocode-maps.yandex.ru/1.x/'
params = {
    'apikey': api_key,
    'format': 'json',
    'geocode': address
}

# Отправка GET-запроса
response = requests.get(base_url, params=params)

# Проверка статуса ответа
if response.status_code == 200:
    # Преобразование ответа в JSON
    data = response.json()
    
    # Извлечение координат из ответа
    coordinates = data['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos']
    
    # Разделение координат на долготу и широту
    longitude, latitude = map(float, coordinates.split())
    
    # Вывод результата
    koord=(f'Координаты для адреса "{address}" составляют: Долгота = {longitude}, Широта = {latitude}')
    print(f'Координаты для адреса "{address}" составляют: Долгота = {longitude}, Широта = {latitude}')
else:
    print(f'Ошибка: {response.status_code}')

Координаты для адреса "410005 индекс Саратовская область Саратов Большая Горная улица № 157 дом" составляют: Долгота = 46.041569, Широта = 51.539473


In [56]:
# Отправляем сообщение на личный почтовый ящик 

import win32com.client as win32
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'Vorobyeva.E.O@sberbank.ru'
mail.Subject = 'Создать заявку на установку'
mail.Body = body_content + koord 
mail.Send()

print('Письмо направлено на ЛПЯ')

Письмо направлено на ЛПЯ
